# Using the DCF to calculate intrinsic value of company.

This notebook calculates the intricsic value of a given company by using the simplest model of discounted cash flow (DFC).
This is done by projecting future cash flows with an obtained growth rate, and afterwards discounting these values to present value using the company's WACC.

The data and projections are based on data obtained from several APIs and webpages (web-scraping)

The script follows the following steps:

Step 1) - Obtaining data (Free Cash Flow, WACC, Estimated Growth, etc.)

Step 2) - Projection of Free Cash Flow

Step 3) - Inserting values into DCF formulas and calculating intrinsic value.

#### Step 1 - Obtaining data

In [305]:
#Importing relevant packages:
import pandas as pd  
import numpy as np  
import yfinance as yf  
import yahoo_fin.stock_info as si
import json
import urllib
from urllib.request import urlopen
import requests as requests
import bs4
from bs4 import BeautifulSoup
import datetime
from datetime import *
import pandas_datareader as web

In [306]:
ticker='AAPL' #The ticker for the company of which one wants to know the intrincsic value

In [307]:
#Using the Financialmodelingprep API to gather Free Cash Flow, Total debt and Cash + Short Term Investments
demo='adc4ac8ffec969a26ed16f5eadb3d077'
url=f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?limit=120&apikey={demo}'

response=urllib.request.urlopen(url)
data=response.read().decode("utf-8")
data=json.loads(data)

data=data[0]
fcf=data['freeCashFlow']

url_1=f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?limit=120&apikey={demo}'

response=urllib.request.urlopen(url_1)
data1=response.read().decode("utf-8")
data1=json.loads(data1)

data1=data1[0]
total_debt=data1['totalDebt']
cash_si=data1['cashAndShortTermInvestments']

In [308]:
#Gets the EPS 5 years for the chosen company
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'} #Header needed to gain access to webpage
url2=(f'https://finviz.com/quote.ashx?t={ticker}')
response = requests.get(url2,headers=headers).text  
table=pd.read_html(response)[4]
eps_5y=table.iloc[5,5]
eps_5y=float(eps_5y.strip('%'))/100

In [309]:
#Using BS to webscrape a page for the weighted average cost of capital (WACC)
url3=(f'https://site.financialmodelingprep.com/weighted-average-cost-of-capital/{ticker}')
returned=requests.get(url3,headers=headers).content
soup=BeautifulSoup(returned,'html.parser')
broth=soup.find(id='wacc')
wacc=broth['value']
wacc=float(wacc)/100

#### Step 2 - Projection of FCF

We have obtained the estimated growth rate of the company for the next 5 years (year 1-5)
Taking a more conservative approach, we assume that the growth rate for year 6-10 is half of this (i.e. EPS_5Y*0.5)
After year 10, we assume that the FCF will grow at a rate of 4% for the next 10 years (here ignoring that it could grow into perpetuity, still an issues im trying to resolve coding-wise)

In [315]:
gfcf_5y=eps_5y
gfcf_10y=gfcf_5y*0.5
gfcf=0.04

print(f'The Free Cash Flow are expected to grow at a rate of {gfcf_5y} for year 1-5.')
print(f'The Free Cash Flow are expected to grow at a rate of {gfcf_10y} for year 6-10.')
print(f'The Free Cash Flow are expected to grow at a rate of {gfcf} for year 11-20.')


The Free Cash Flow are expected to grow at a rate of 0.1485 for year 1-5.
The Free Cash Flow are expected to grow at a rate of 0.07425 for year 6-10.
The Free Cash Flow are expected to grow at a rate of 0.04 for year 11-20.


In [311]:
#Below is two for loops, that iterates through the given year, and calculates its projected free cash flow and then discounts it to present value
discount_fcf_list=[]

for n in range(1,6):
    projected_fcf=fcf*(1+gfcf_5y)**n
    discounted_fcf=projected_fcf/(1+wacc)**n
    discount_fcf_list.append(discounted_fcf)


for n in range(6,11):
    fcf_5=fcf*(1+gfcf_5y)**5
    projected_fcf=fcf_5*(1+gfcf_10y)**(n-5)
    discounted_fcf=projected_fcf/(1+wacc)**n
    discount_fcf_list.append(discounted_fcf)

for n in range(11,21):
    fcf_10=fcf_5*(1+gfcf_10y)**5
    projected_fcf=fcf_10*(1+gfcf)**(n-10)
    discounted_fcf=projected_fcf/(1+wacc)**n
    discount_fcf_list.append(discounted_fcf)

#### Step 3 - Inserting values into DCF formulas and calculating intrinsic value.

In [312]:
#Gets the number of outstanding shares
si_data=si.get_quote_data(ticker)
shares_outstanding=si_data['sharesOutstanding']

In [313]:
#Sums up all the future free cash flow
total_fcf=sum(discount_fcf_list)

In [316]:
#Inserting our values into the (simple) DCF formuka
p_0=(total_fcf+cash_si-total_debt)/shares_outstanding
p_0

134.02797708705

The final return of the script is the estimated instrinsic value of a given company.
It most certainly is a simplified valuation model, that draws upon several assumptions that might not always be true in reality.

The model will most likely give the best estimations when working with blue chip stocks, that are stable and where growth is stable and somewhat predictable (I.e not growth-companies like Tesla).